# Spectral ROI - Example Usage

This notebook demonstrates how to use the `spectral-roi` package to extract spectral data from TIFF stacks.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

# Import spectral_roi functions
import spectral_roi
from spectral_roi import (
    extract_roi_spectra,
    extract_full_image_spectrum,
    extract_pixel_spectra,
    extract_grid_spectra,
)

# Import interactive tools (requires plotly and ipywidgets)
try:
    from spectral_roi import interactive
    INTERACTIVE_AVAILABLE = True
except ImportError:
    INTERACTIVE_AVAILABLE = False
    print("Interactive tools not available. Install with: pip install save-roi[interactive]")

## Example 1: Extract Spectra Using ImageJ ROI File

This is the most common use case - you have a TIFF stack and an ImageJ ROI file (or ROI zip file).

## Example 0: Interactive ROI Selection Tool ⭐ NEW

The interactive ROI selector provides an ImageJ-like interface for drawing, managing, and extracting ROIs directly in Jupyter notebooks.

**Note:** This requires the interactive dependencies. Install with:
```bash
pip install save-roi[interactive]
```

In [ ]:
# Launch the interactive ROI selector
# This will display an interactive widget below this cell
if INTERACTIVE_AVAILABLE:
    tiff_path = "image.tiff"
    
    # Option 1: Simple launch
    selector = interactive.launch_interactive_tool(tiff_path)
    
    # The tool will display:
    # - Interactive Plotly heatmap of the summed TIFF stack
    # - Drawing tools to create ROIs (rectangle, circle, polygon)
    # - ROI list and management controls
    # - Buttons to save/load ROIs and extract spectra
    
    # Option 2: Programmatic control (example - don't run both)
    # selector = interactive.InteractiveROISelector(tiff_path)
    # selector.create_grid_rois(grid_size=4)  # Create 4x4 grid
    # selector.save_rois("my_rois.zip")
    # results = selector.extract_spectra(output_dir="./results")
else:
    print("Interactive tools not available. Install with: pip install save-roi[interactive]")

In [ ]:
# Define file paths
tiff_path = "image.tiff"
roi_path = "ROI2.zip"

# Extract spectra for all ROIs
results = extract_roi_spectra(
    tiff_path=tiff_path,
    roi_path=roi_path,
    save_csv=True  # This will save CSV files to image_ROI_Spectra/ folder
)

print(f"Found {len(results)} ROI(s)")
print("ROI names:", list(results.keys()))

In [ ]:
# View the data for the first ROI
first_roi_name = list(results.keys())[0]
df = results[first_roi_name]

print(f"\nData for {first_roi_name}:")
print(df.head(10))

In [ ]:
# Plot the spectrum
plt.figure(figsize=(10, 6))

for roi_name, df in results.items():
    plt.errorbar(
        df['stack'],
        df['counts'],
        yerr=df['err'],
        label=roi_name,
        marker='o',
        capsize=3
    )

plt.xlabel('Stack Number (Slice)')
plt.ylabel('Counts')
plt.title('Spectral Profile by ROI')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Example 2: Extract Spectrum for Entire Image (No ROI)

If you don't have an ROI file, you can analyze the entire image.

In [ ]:
# Extract spectrum for full image
full_spectrum = extract_full_image_spectrum(
    tiff_path=tiff_path,
    save_csv=True
)

print(full_spectrum.head(10))

In [ ]:
# Plot full image spectrum
plt.figure(figsize=(10, 6))
plt.errorbar(
    full_spectrum['stack'],
    full_spectrum['counts'],
    yerr=full_spectrum['err'],
    marker='o',
    capsize=3,
    label='Full Image'
)
plt.xlabel('Stack Number (Slice)')
plt.ylabel('Counts')
plt.title('Full Image Spectral Profile')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Example 3: Extract Spectra Using Grid-Based Analysis

Analyze the image using a grid pattern (e.g., 4x4 pixel blocks).

In [ ]:
# Extract spectra for 4x4 pixel grid
# Note: This will create many CSV files, so we'll use a smaller grid or disable saving
grid_results = extract_grid_spectra(
    tiff_path=tiff_path,
    grid_size=8,  # 8x8 pixel blocks
    output_dir="image_ROI_Spectra/grid_8x8",
    save_csv=True
)

print(f"Generated {len(grid_results)} grid spectra")

In [ ]:
# Plot a few grid spectra
plt.figure(figsize=(12, 6))

# Plot first 5 grid cells
for idx, (grid_name, df) in enumerate(list(grid_results.items())[:5]):
    plt.plot(
        df['stack'],
        df['counts'],
        marker='o',
        label=grid_name,
        alpha=0.7
    )

plt.xlabel('Stack Number (Slice)')
plt.ylabel('Counts')
plt.title('Grid-Based Spectral Profiles (First 5 cells)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Example 4: Pixel-by-Pixel Analysis with Stride

Extract spectra for individual pixels. Use stride to sample every Nth pixel to reduce output size.

In [ ]:
# Extract pixel spectra with stride=10 (every 10th pixel)
pixel_results = extract_pixel_spectra(
    tiff_path=tiff_path,
    stride=10,
    output_dir="image_ROI_Spectra/pixels_stride10",
    save_csv=False  # Disable saving to avoid too many files
)

print(f"Generated {len(pixel_results)} pixel spectra")

In [ ]:
# Plot a few pixel spectra
plt.figure(figsize=(12, 6))

# Plot first 5 pixels
for idx, (pixel_name, df) in enumerate(list(pixel_results.items())[:5]):
    plt.plot(
        df['stack'],
        df['counts'],
        marker='o',
        label=pixel_name,
        alpha=0.7
    )

plt.xlabel('Stack Number (Slice)')
plt.ylabel('Counts')
plt.title('Pixel-by-Pixel Spectral Profiles (First 5 pixels)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Example 5: Custom Output Directory

Specify a custom location for output CSV files.

In [ ]:
# Save results to a custom directory
custom_results = extract_roi_spectra(
    tiff_path=tiff_path,
    roi_path=roi_path,
    output_dir="./my_custom_results",
    save_csv=True
)

print(f"Results saved to: ./my_custom_results")

## Example 6: Working with Results in Memory

You can disable CSV saving and work with the results directly in memory.

In [ ]:
# Get results without saving CSV files
results_no_save = extract_roi_spectra(
    tiff_path=tiff_path,
    roi_path=roi_path,
    save_csv=False
)

# Do custom analysis
for roi_name, df in results_no_save.items():
    total_counts = df['counts'].sum()
    mean_counts = df['counts'].mean()
    print(f"{roi_name}: Total={total_counts:.1f}, Mean={mean_counts:.1f}")

## Summary

The `spectral-roi` package provides flexible options for extracting spectral data:

0. **Interactive ROI Selection** ⭐ NEW: Visual interface for drawing and managing ROIs in Jupyter (requires `pip install save-roi[interactive]`)
1. **ROI-based**: Use ImageJ ROI files for specific regions
2. **Full image**: Analyze the entire image
3. **Grid-based**: Systematic grid pattern analysis
4. **Pixel-based**: Individual pixel analysis with optional stride

All methods produce CSV files with the same structure:
- `stack`: Slice number (1-indexed)
- `counts`: Total counts in the region
- `err`: Error (square root of counts)

For command-line usage, try:
```bash
save-roi -t image.tiff -i  # Launch interactive tool
save-roi -t image.tiff -r ROI2.zip  # Use ROI file
```